Feature Engineering
Process of selecting and transforming training data into new variables that are used as inputs for the model.

Domain knowledge, statistics, visualisation
Missing values, dirty data, outliers (remove or replace), skew, normalization

EDA:
Attributes
date - Date of the sale data. There are no holiday effects or store closures.
store - Store ID
item - Item ID
sales - Number of items sold at a particular store on a particular date.

Entries (Train / Test) : 913000 / 45000
Stores (Train / Test) : 1 - 10 / 1 - 10
Items (Train / Test) : 1 - 50 / 1 - 50
Dates (Train / Test) : 2013-01-01 - 2017-12-31 / 2018-01-01 - 2018-03-31

no obvious outliers, data follows seasonal pattern with linear growth


Questions to consider:
What's the best way to deal with seasonality? 
Should stores be modeled separately, or can you pool them together? 

Evaluation metric: SMAPE
Facebook Prophet:
Pros:

Quite easy to use
Allows specifying multiple seasonalities
Allows specifying special events
Can compute quick MAP and slow but accurate Bayesian estimates
Provides methods for basic plotting out of the box

Cons:

Can only treat univariate time series
Assumes Gaussian priors
Does not provide methods to tune hyper-parameters out of the box (for example, seasonality and trend flexibility priors)

TASKS:
optuna to optimise hyperparameters
tsfresh for feature engineering
dtreeviz to visualise xgboost model
Implement fbprophet
Custom SMAPE eval metric for model.fit


In [ ]:
!pip install scikit-learn
!pip install xgboost
!pip install pystan 
!pip install prophet
!pip install featuretools
!pip install tsfresh
!git clone https://github.com/jeslago/epftoolbox.git

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
import featuretools as ft
import tsfresh as ts
from prophet import Prophet
from epftoolbox.evaluation import sMAPE

In [77]:
import os
os.chdir('epftoolbox')
!pip install .
os.chdir('..')

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   store   913000 non-null  int64 
 2   item    913000 non-null  int64 
 3   sales   913000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 27.9+ MB


In [4]:
df_train.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [5]:
df_train['date'] = pd.to_datetime(df_train['date'], utc=True)
#df_train = pd.DatetimeIndex(df_train['date'])

df_test['date'] = pd.to_datetime(df_test['date'], utc=True)
#df_test = pd.DatetimeIndex(df_test['date'])

In [6]:
# create features
df_train['year'] = df_train['date'].dt.year #pd.DatetimeIndex(df_train['ArrivalDate']).year
df_train['month'] = df_train['date'].dt.month
df_train['dayofyear'] = df_train['date'].dt.dayofyear
df_train['dayofweek'] = df_train['date'].dt.dayofweek
df_train['quarter'] = df_train['date'].dt.quarter

df_test['year'] = df_test['date'].dt.year
df_test['month'] = df_test['date'].dt.month
df_test['dayofyear'] = df_test['date'].dt.dayofyear
df_test['dayofweek'] = df_test['date'].dt.dayofweek
df_test['quarter'] = df_test['date'].dt.quarter
#df_test.head()

# Lag features: using past value of forecast var as a feature
df_train.index = pd.DatetimeIndex(df_train['date'])
target_map = df_train['sales'].to_dict()
df_train['lag_feature'] = (df_train.index - pd.Timedelta('364 days')).map(target_map)


In [7]:
#Check lag features
df_train['lag_feature'].tail()

date
2017-12-27 00:00:00+00:00    43.0
2017-12-28 00:00:00+00:00    68.0
2017-12-29 00:00:00+00:00    63.0
2017-12-30 00:00:00+00:00    64.0
2017-12-31 00:00:00+00:00    69.0
Name: lag_feature, dtype: float64

In [8]:
df_train = df_train.drop(['date'], axis=1)
df_test = df_test.drop(['date'], axis=1)
df_train.columns

Index(['store', 'item', 'sales', 'year', 'month', 'dayofyear', 'dayofweek',
       'quarter', 'lag_feature'],
      dtype='object')

In [9]:
features_list = ['store', 'item', 'year', 'month', 'dayofweek', 'dayofyear', 'quarter','lag_feature']
forecast_var = ['sales']

# Do cross validation with 1 year forecast horizon
tss = TimeSeriesSplit(n_splits = 5, test_size = 365)
df_train = df_train.sort_index()

for train_index, validation_index in tss.split(df_train):
    df_train_split = df_train.iloc[train_index]
    df_validation = df_train.iloc[validation_index]
    
X_train = df_train_split[features_list]
X_validation = df_validation[features_list]
y_train = df_train_split[forecast_var]
y_validation = df_validation[forecast_var]

#X_train, X_validation, y_train, y_validation = train_test_split(df_train[features_list],df_train[forecast_var], train_size=0.8)

#X_test = df_test[features_list]
#y_test = df_test[forecast_var]

In [29]:
# Create model object
"""
Parameters to test: learning rate=0.01, max depth=12, reg_lambda=0.5 

1. 1000 estimators, learning_rate = 0.001, booster = 'gbtree',rmse:29.75191
2. 1000 estimators, learning_rate = 0.001, booster = 'gbtree',rmse:60.14270 w TimeSeriesSplit
3. 1000 estimators, learning_rate = 0.001, booster = 'gbtree',rmse:28.29920 w lag feature
4. 5000 estimators, learning_rate = 0.001, booster = 'gbtree',rmse:13.17542 w lag feature
5. 1000 estimators, learning_rate = 0.001, booster = 'dart',rmse:28.28290 w lag feature. Extremely slow training
6. 5000 estimators, learning_rate = 0.001, booster = 'gbtree',rmse:8.90808 w max_depth = 12
7. 1000 estimators, learning_rate = 0.01, booster = 'gbtree',rmse:7.87088 w max_depth = 12, starts overfitting with more estimators
8. 1000 estimators, learning_rate = 0.01, booster = 'gbtree', max_depth = 12, reg_lambda = 0.5, rmse:7.85207

"""
#%%capture output
model = xgb.XGBRegressor(n_estimators = 1000, early_stopping_rounds = 50, learning_rate = 0.001, sample_type = 'weighted', booster = 'dart')
# model.load_model('model.json')
model.fit(X_train, y_train, eval_set = [(X_validation,y_validation)], verbose = 10)


[0]	validation_0-rmse:56.78697
[10]	validation_0-rmse:56.36713
[20]	validation_0-rmse:55.95027
[30]	validation_0-rmse:55.52443
[40]	validation_0-rmse:55.11365
[50]	validation_0-rmse:54.71021
[60]	validation_0-rmse:54.31031
[70]	validation_0-rmse:53.92092
[80]	validation_0-rmse:53.52628
[90]	validation_0-rmse:53.13848
[100]	validation_0-rmse:52.75935
[110]	validation_0-rmse:52.35489
[120]	validation_0-rmse:51.91417
[130]	validation_0-rmse:51.48708
[140]	validation_0-rmse:51.05330
[150]	validation_0-rmse:50.63382
[160]	validation_0-rmse:50.20793
[170]	validation_0-rmse:49.78800
[180]	validation_0-rmse:49.36381
[190]	validation_0-rmse:48.95615
[200]	validation_0-rmse:48.53909
[210]	validation_0-rmse:48.14736
[220]	validation_0-rmse:47.75231
[230]	validation_0-rmse:47.35370
[240]	validation_0-rmse:46.97091
[250]	validation_0-rmse:46.58177
[260]	validation_0-rmse:46.20710
[270]	validation_0-rmse:45.83441
[280]	validation_0-rmse:45.46320
[290]	validation_0-rmse:45.10608


KeyboardInterrupt: 

In [20]:
df_feature_importance = pd.DataFrame(index = model.get_booster().feature_names, data = model.feature_importances_, columns = ['Importance'])
print(model.get_booster().feature_names)
df_feature_importance.sort_values('Importance')

['store', 'item', 'year', 'month', 'dayofweek', 'dayofyear', 'quarter', 'lag_feature']


,Importance
quarter,0.000000
year,0.016784
dayofyear,0.017603
dayofweek,0.051125
month,0.099540
store,0.135891
lag_feature,0.168955
item,0.510102


In [26]:
# Slow for complex models
y_train_pred = pd.DataFrame(model.predict(X_train))
y_validation_pred = pd.DataFrame(model.predict(X_validation))
# Convert index from Range64 to Int64 
y_train_pred.index = list(y_train_pred.index)
y_validation_pred.index = list(y_validation_pred.index)

y_train.reset_index(drop=True, inplace=True)
y_train_pred_real = pd.concat([y_train,y_train_pred],axis=1)
y_validation.reset_index(drop=True, inplace=True)
y_validation_pred_real = pd.concat([y_validation,y_validation_pred],axis=1)

print(y_train_pred_real.head())
print(y_validation_pred_real.head())

   sales          0
0     13  11.279771
1     26  25.576149
2     27  20.528736
3     54  44.011692
4     35  38.287983
   sales          0
0     35  23.416321
1     17  17.925344
2     22  22.780067
3     21  20.664536
4     19  20.180243


In [27]:
# custom evaluation metric, target SMAPE <0.02
"""
1. Train set sMAPE:  0.43540993893878704 Validation set sMAPE:  0.4353321748664415
2. Train set sMAPE:  0.43428628486970966 Validation set sMAPE:  0.8993101261777344
3. Train set sMAPE:  0.43886955493496904 Validation set sMAPE:  0.4591814227399213
4. Train set sMAPE:  0.21175586516844072 Validation set sMAPE:  0.2135402045304419
5. None
6. Train set sMAPE:  0.13079433706697108 Validation set sMAPE:  0.15268113263167146
7. Train set sMAPE:  0.11717552685158845 Validation set sMAPE:  0.13378531448480144
"""
train_sMAPE = sMAPE(p_pred=y_train_pred, p_real=y_train) 
validation_sMAPE = sMAPE(p_pred=y_validation_pred, p_real=y_validation) 
print("Train set sMAPE: ", train_sMAPE)
print("Validation set sMAPE: ", validation_sMAPE)

Train set sMAPE:  0.11763192366525083
Validation set sMAPE:  0.13544394278689448


In [28]:
# Save model
model.save_model('model_9.json')

In [ ]:
# Use model to predict on test data
df_test['sales_predicted'] = model.predict(X_test)
df_test.head()

There are in general two ways that you can control overfitting in XGBoost:<br>

The first way is to directly control model complexity.<br>

This includes max_depth, min_child_weight and gamma.<br>

The second way is to add randomness to make training robust to noise.<br>

This includes subsample and colsample_bytree.<br>

You can also reduce stepsize eta. Remember to increase num_round when you do so.
<br><br>
There’s a parameter called tree_method, set it to hist or gpu_hist for faster computation.
<br><br>
gbtree parameters:
<br>
eta [default=0.3, alias: learning_rate]<br>

Step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative.<br>
max_depth [default=6]<br>

Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit. 0 indicates no limit on depth. Beware that XGBoost aggressively consumes memory when training a deep tree.<br>
lambda [default=1, alias: reg_lambda]<br>

L2 regularization term on weights. Increasing this value will make model more conservative.<br>

The booster dart inherits gbtree booster, so it supports all parameters that gbtree does, such as eta, gamma, max_depth etc.
<br><br>
Additional parameters are noted below:<br>

sample_type: type of sampling algorithm.<br>

uniform: (default) dropped trees are selected uniformly.<br>

weighted: dropped trees are selected in proportion to weight.<br>

normalize_type: type of normalization algorithm.<br>

tree: (default) New trees have the same weight of each of dropped trees.<br>
forest: New trees have the same weight of sum of dropped trees (forest).<br>
rate_drop: dropout rate.<br>

range: [0.0, 1.0]<br>

skip_drop: probability of skipping dropout.<br>

If a dropout is skipped, new trees are added in the same manner as gbtree.<br>

range: [0.0, 1.0]<br><br>

